In [25]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zgnbyktanr/turkish-psychological-classification-for-nlp")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/zgnbyktanr/turkish-psychological-classification-for-nlp/versions/1


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import re
from imblearn.over_sampling import SMOTE
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
# nltk stopwords yükleyelim
nltk.download('stopwords')
stop_words = set(stopwords.words('turkish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
import os

# Dosya yolundaki dosyaları listeleyin
dataset_path = '/root/.cache/kagglehub/datasets/zgnbyktanr/turkish-psychological-classification-for-nlp/versions/1'
files = os.listdir(dataset_path)
print(files)



['dataset.csv', 'test.xlsx']


In [32]:
import pandas as pd

# Doğru dosya yolunu kullanarak CSV dosyasını okuyun
file_path = '/root/.cache/kagglehub/datasets/zgnbyktanr/turkish-psychological-classification-for-nlp/versions/1/dataset.csv'
df = pd.read_csv(file_path)

# İlk birkaç satırı inceleyin
print(df.head())



                                                text  label
0  İlişkilerimde aşırı derecede idealize edip son...      6
1  Bir gece uykumda yürüyüp dolaptan kıyafetlerim...      2
2  Kumar borçlarım yüzünden evliliğim tehlikede. ...      4
3  Uykum çok hafif ve en ufak bir sesle bile uyan...      2
4  Kumar borçlarımı ödemek için çaldığım paraları...      4


In [33]:
# 1. Veri Ön İşleme
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

In [34]:
# 2. Özellik Çıkarımı
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Unigrams ve Bigrams
X = tfidf.fit_transform(df['cleaned_text']).toarray()
y = df['label']

In [35]:
# Veri setini ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [36]:
# SMOTE uygulayalım
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [38]:
# Lojistik Regresyon Modeli
logistic_model = LogisticRegression()
logistic_model.fit(X_train_resampled, y_train_resampled)
y_pred_logistic = logistic_model.predict(X_test)
print("Lojistik Regresyon Doğruluk Skoru:", accuracy_score(y_test, y_pred_logistic))
print("Lojistik Regresyon Sınıflandırma Raporu:\n", classification_report(y_test, y_pred_logistic))

Lojistik Regresyon Doğruluk Skoru: 0.7993079584775087
Lojistik Regresyon Sınıflandırma Raporu:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.88      1.00      0.93        14
           2       0.95      0.95      0.95        20
           3       1.00      1.00      1.00         8
           4       0.82      0.90      0.86        52
           5       0.70      0.70      0.70        10
           6       0.90      0.60      0.72        15
           7       0.67      0.40      0.50         5
           8       0.78      0.78      0.78        18
           9       0.92      0.71      0.80        17
          10       0.64      0.68      0.66        34
          11       0.84      0.79      0.82        34
          12       1.00      0.43      0.60         7
          13       0.59      0.80      0.68        20
          14       0.75      0.88      0.81        17
          15       1.00      0.75      

In [39]:
# Sinir Ağı Modeli
input_dim = X_train_resampled.shape[1]  # TF-IDF özellik boyutu

nn_model = Sequential([
    Dense(128, activation='relu', input_dim=input_dim),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y)), activation='softmax')  # Çoklu sınıf için softmax kullanılıyor
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
# Modeli eğitelim
nn_model.fit(X_train_resampled, y_train_resampled, epochs=20, batch_size=16, validation_split=0.5, verbose=1)

# Test veri seti ile tahmin yapalım
y_pred_nn = np.argmax(nn_model.predict(X_test), axis=1)

print("Sinir Ağı Doğruluk Skoru:", accuracy_score(y_test, y_pred_nn))
print("Sinir Ağı Sınıflandırma Raporu:\n", classification_report(y_test, y_pred_nn))

Epoch 1/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9936 - loss: 0.0123 - val_accuracy: 1.0000 - val_loss: 4.5906e-05
Epoch 2/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9945 - loss: 0.0209 - val_accuracy: 1.0000 - val_loss: 4.4946e-05
Epoch 3/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9979 - loss: 0.0092 - val_accuracy: 1.0000 - val_loss: 4.2521e-05
Epoch 4/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9970 - loss: 0.0165 - val_accuracy: 1.0000 - val_loss: 5.4653e-05
Epoch 5/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9960 - loss: 0.0146 - val_accuracy: 1.0000 - val_loss: 5.0438e-05
Epoch 6/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9968 - loss: 0.0078 - val_accuracy: 1.0000 - val_loss: 4.3611e-05
Epoch 7/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9921 - loss: 0.0165 - val_accuracy: 1.0000 - val_loss: 7.2167e-05
Epoch 8/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9949 - loss: 0.016

In [41]:
# Kullanıcıdan cümle alıp tahmin yapma
label_dict = {
    0: "Suicidal Thoughts",
    1: "Eating Disorders",
    2: "Sleep Disorders",
    3: "Sexual Disorders",
    4: "Addictions",
    5: "Anger Control Disorders",
    6: "Borderline",
    7: "Psychosomatic Disorders",
    8: "OCD (Obsessive-Compulsive Disorder)",
    9: "Behavioral Disorders in Children",
    10: "Depression and Related Disorders",
    11: "Family and Relationship Issues",
    12: "Sports Psychology",
    13: "Attention Deficit and Hyperactivity Disorder (ADHD)",
    14: "Trauma",
    15: "Paraphilic Disorders"
}


In [45]:
def classify_sentences():
    sentences = []
    while True:
        print("\nSeçenekler:")
        print("1. Cümle Gir")
        print("2. Sonucu Gör")
        print("3. Çıkış")
        choice = input("Bir seçenek girin: ")

        if choice == '1':
            sentence = input("Lütfen bir cümle girin: ")
            sentences.append(sentence)
            print(f"Cümle eklendi: {sentence}")
        elif choice == '2':
            if not sentences:
                print("Lütfen önce cümle girin!")
                continue
            cleaned_sentences = [clean_text(sentence) for sentence in sentences]
            vectorized_sentences = tfidf.transform(cleaned_sentences).toarray()

            logistic_predictions = logistic_model.predict(vectorized_sentences)
            nn_predictions = np.argmax(nn_model.predict(vectorized_sentences), axis=1)

            print("\nSonuçlar:")
            for i, sentence in enumerate(sentences):
                logistic_class = label_dict.get(logistic_predictions[i], "Bilinmeyen Rahatsızlık")
                nn_class = label_dict.get(nn_predictions[i], "Bilinmeyen Rahatsızlık")
                print(f"Cümle {i+1}: '{sentence}'")
                print(f"  Lojistik Regresyon Tahmini: {logistic_predictions[i]} ({logistic_class})")
                print(f"  Sinir Ağı Tahmini: {nn_predictions[i]} ({nn_class})")

            sentences.clear()
        elif choice == '3':
            print("\u00c7ıkılıyor...")
            break
        else:
            print("Geçersiz seçenek. Lütfen tekrar deneyin.")

classify_sentences()



Seçenekler:
1. Cümle Gir
2. Sonucu Gör
3. Çıkış
Bir seçenek girin: intihar etmek istiyorum
Geçersiz seçenek. Lütfen tekrar deneyin.

Seçenekler:
1. Cümle Gir
2. Sonucu Gör
3. Çıkış
Bir seçenek girin: 1
Lütfen bir cümle girin: intihar etmek istiyorum
Cümle eklendi: intihar etmek istiyorum

Seçenekler:
1. Cümle Gir
2. Sonucu Gör
3. Çıkış
Bir seçenek girin: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Sonuçlar:
Cümle 1: 'intihar etmek istiyorum'
  Lojistik Regresyon Tahmini: 0 (Suicidal Thoughts)
  Sinir Ağı Tahmini: 10 (Depression and Related Disorders)

Seçenekler:
1. Cümle Gir
2. Sonucu Gör
3. Çıkış
Bir seçenek girin: 3
Çıkılıyor...
